In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import imageio
import torch
from src import MotionDataset

from src.full_dataset import SNACMotionDataset
from IPython.display import Audio
import matplotlib.pyplot as plt

/home/xabieririzar/liveportrait-tokenizer/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from src import TokenizerModule

In [ ]:
ds = MotionDataset("dataset", split="eval", val_split=0.1, seed=2, compute_stats=False)

Loading precomputed statistics from dataset/stats_all.pkl
Loaded feature-wise statistics successfully
Loaded 2045 eval samples


In [7]:
audio = full_ds[134]['audio']
Audio(audio.squeeze().cpu().numpy(), rate=24000)

In [22]:
# sample = ds.resample_item(ds[134], 24)
sample = ds[134]

In [23]:
# sample = ds[134] 

# pickle_path = sample['metadata']['pickle_path']
# vid_id = pickle_path.split("/")[-1].split(".")[0]
# vid_path = f"dataset/train/{vid_id}.mp4"

# # Read the first frame from the video and display it
# frame = imageio.get_reader(vid_path).get_data(0)
# plt.imshow(frame)
# plt.show()

In [24]:
# tokenizer = TokenizerModule(tokenizer_config="configs/tokenizer_config.yaml")
tokenizer = TokenizerModule.from_pretrained("InternalCan/tokenizer_module")

Loading precomputed statistics from dataset/stats_all.pkl
Loaded feature-wise statistics successfully
Loaded 2045 test samples
['exp']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['exp']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['c_eyes_lst', 'c_lip_lst', 'kp', 't', 'x_s']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256
['R', 'scale']
FSQ Config:
Levels: 4
Output Emb Width: 4
Num Quantizers: 1
Using FSQ
Codebook size: 256


In [25]:
features = tokenizer.sample_to_features(sample)

In [26]:
# features = tokenizer.pickle_to_features("female_24fps.pkl")

In [28]:
features.shape

torch.Size([1, 147, 205])

In [29]:
indices = tokenizer.features_to_codes(features)

In [30]:
# torch.save(indices, "xabi_24fps.pt")

In [31]:
reconstr_feats = tokenizer.codes_to_features(indices)

c_eyes_lst 0 : 2
c_lip_lst 2 : 3
kp 3 : 66
t 66 : 69
x_s 69 : 132
exp 0 : 63
R 0 : 9
scale 9 : 10


In [36]:
output = tokenizer.features_to_pickle(reconstr_feats, pickle_path=sample['metadata']['pickle_path'])

dataset/pickles/WQvT1_tQDhg_22_reconstructed.pkl


In [37]:
new_path = "dataset/pickles/WQvT1_tQDhg_22_reconstructed.pkl"
# new_path = "female_24fps_reconstructed.pkl"

In [38]:
# # Replace reconstructed features with original ones for a specific range
# feat = 'exp'

# feat_range = (0, 21)

# denormalized_sample = ds.denormalize_features(sample[feat], feat)

# for i in range(output['n_frames']):
#     output['motion'][i][feat][:, feat_range[0]:feat_range[1]] = 0

In [39]:
!python inference.py -d {new_path} -s assets/examples/source/reconstructed.png
# !python inference.py -d {new_path} -s assets/examples/source/reconstructed.png

[19:57:48] Load appearance_feature_extractor from    ]8;id=241099;file:///home/xabieririzar/liveportrait-tokenizer/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=91456;file:///home/xabieririzar/liveportrait-tokenizer/src/live_portrait_wrapper.py#46\46]8;;\
           /home/xabieririzar/liveportrait-tokenizer                            
           /pretrained_weights/liveportrait/base_mod                            
           els/appearance_feature_extractor.pth                                 
           done.                                                                
[19:57:49] Load motion_extractor from                ]8;id=714396;file:///home/xabieririzar/liveportrait-tokenizer/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=132910;file:///home/xabieririzar/liveportrait-tokenizer/src/live_portrait_wrapper.py#49\49]8;;\
           /home/xabieririzar/liveportrait-tokenizer                            
           /pretrained_weigh

In [ ]:
reader = imageio.get_reader("xabi.mp4")

In [ ]:
reader.get_meta_data()

In [ ]:
# Process full video and save as xabi_scaled.mp4 with 512x512 resolution
import imageio
from PIL import Image
import numpy as np
import subprocess
import os

# Read the original video
reader = imageio.get_reader("amu.mp4")
writer = imageio.get_writer("amu_scaled_temp.mp4", fps=reader.get_meta_data()['fps'])

# Get first frame to calculate scaling parameters
first_frame = reader.get_data(0)
pil_frame = Image.fromarray(first_frame)
width, height = pil_frame.size

# Calculate scale factor to make the smaller dimension 512
scale = 512 / min(width, height)
new_width = int(width * scale)
new_height = int(height * scale)

# Calculate crop coordinates to center the 512x512 crop
left = (new_width - 512) // 2
top = (new_height - 512) // 2
right = left + 512
bottom = top + 512

# Process all frames
for frame in reader:
    # Convert to PIL Image
    pil_frame = Image.fromarray(frame)
    # Resize maintaining aspect ratio
    resized_frame = pil_frame.resize((new_width, new_height), Image.LANCZOS)
    # Crop to 512x512
    cropped_frame = resized_frame.crop((left, top, right, bottom))
    # Convert back to numpy array and write to video
    processed_frame = np.array(cropped_frame)
    writer.append_data(processed_frame)

# Close reader and writer
reader.close()
writer.close()

# Use ffmpeg to combine the processed video with original audio
subprocess.run([
    'ffmpeg', '-i', 'amu_scaled_temp.mp4', '-i', 'amu.mp4', 
    '-c:v', 'copy', '-c:a', 'aac', '-map', '0:v:0', '-map', '1:a:0',
    '-y', 'amu_scaled.mp4'
], check=True)

# Remove temporary file
os.remove('amu_scaled_temp.mp4')

# Also save first frame as jpg for reference
first_frame_processed = imageio.get_reader("amu_scaled.mp4").get_data(0)
Image.fromarray(first_frame_processed).save("amu_first_frame.jpg")

# Display first frame
plt.imshow(first_frame_processed)
plt.show()

In [ ]:
tokenizer.save_pretrained("tokenizer_module")
tokenizer.push_to_hub("InternalCan/tokenizer_module")
